In [1]:
#Import necessary libraries
import os
import ee
import geemap
import ipyleaflet
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import statsmodels.api as sm
import pandas as pd
from IPython.display import HTML, display
import random
import json
import time
import neighborhood_predictor_variables
import gee_classifiers
num_seed=30
random.seed(num_seed)

In [2]:
#Initialize earth engine
ee.Initialize()


In [3]:
#Define functions for mapping MapBiomas and simplifying the legend
coverage_palette =  ['ffffff', '129912', '1f4423', '006400', '00ff00', '687537', '76a5af', '29eee4', 
                     '77a605', '935132', 'bbfcac', '45c2a5', 'b8af4f', 'f1c232', 'ffffb2', 'ffd966', 
                     'f6b26b', 'f99f40', 'e974ed', 'd5a6bd', 'c27ba0', 'fff3bf', 'ea9999', 'dd7e6b', 
                     'aa0000', 'ff99ff', '0000ff', 'd5d5e5', 'dd497f', 'b2ae7c', 'af2a2a', '8a2be2', 
                     '968c46', '0000ff', '4fd3ff']


simple_palette = ['129912','BBFCAC','FFFFB2','EA9999','0000FF','D5D5E5']
statesViz = {'min': 0, 'max': 34, 'palette': coverage_palette};
simpleStatesViz = {'min': 1, 'max': 6, 'palette': simple_palette};

change_detection_palette = ['df07b5','0741df']
changeDetectionViz = {'min': 0, 'max': 1, 'palette': change_detection_palette};

#Load in mapbiomas
mapbiomas_states=ee.Image('projects/mapbiomas-workspace/public/collection4_1/mapbiomas_collection41_integration_v1')
states_mask = mapbiomas_states.mask()

#Define function to convert hierarchical legend to simplest form
def simplify_legend(bandName):
    simplify = mapbiomas_states.expression(
        '(b0 >=1)  && (b0<10) ? 1 :'+
        '((b0>=10) && (b0<14)) || (b0==32) || (b0==29) ? 2 :'+
        '((b0>=18) && (b0<22)) || ((b0>=14)&&(b0<16)) ? 3 :'+
        '((b0>=22) && (b0<26)) || (b0==30) ? 4 :'+
        '(b0==26) || (b0==33) || (b0==31) ? 5 : 6', 
        {
          'b0': mapbiomas_states.select([bandName])
        })
    simplify = simplify.select(['constant'],[bandName])
    return simplify

#Select bands we are interested in
bandList = ['classification_1985', 'classification_1986', 'classification_1987', 'classification_1988', 
             'classification_1989', 'classification_1990', 'classification_1991', 'classification_1992', 
             'classification_1993', 'classification_1994', 'classification_1995', 'classification_1996', 
             'classification_1997', 'classification_1998', 'classification_1999', 'classification_2000', 
             'classification_2001', 'classification_2002', 'classification_2003', 'classification_2004', 
             'classification_2005', 'classification_2006', 'classification_2007', 'classification_2008', 
             'classification_2009', 'classification_2010', 'classification_2011', 'classification_2012', 
             'classification_2013', 'classification_2014', 'classification_2015', 'classification_2016', 
             'classification_2017', 'classification_2018']
bandsEEList = ee.List(bandList) 
states_simple = ee.ImageCollection(bandsEEList.map(simplify_legend)).toBands()
states_simple = states_simple.select(states_simple.bandNames(),bandsEEList)
states_simple = states_simple.updateMask(states_mask)
states_simple = states_simple.set(ee.Dictionary({'min_value':1,'max_value':5}))
#states_simple is now an iamge where each band corresponds to the land cover class for the band name year

#Map one year to check it out!
Map1 = geemap.Map(center=[-9,-51], zoom=4)
Map1.addLayer(mapbiomas_states.select('classification_2018'),statesViz,name='Original MapBiomas')
Map1.addLayer(states_simple.select('classification_2018'),simpleStatesViz,name='Simplified MapBiomas')
display(Map1)


Map(center=[-9, -51], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=F…

In [4]:
#Save 30 meter projection
#Save 30 meter projection
projection_30m = mapbiomas_states.projection().getInfo()
crs = projection_30m.get('crs')
crsTransform = projection_30m.get('transform')
print(crs, crsTransform)


EPSG:4326 [0.0002694945852358564, 0, -74.54381924374933, 0, -0.0002694945852358564, 6.792611020869763]


In [5]:
#Convert long band names to short band names
intBandNames = ['1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', 
             '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',  '2005', '2006', '2007', '2008', 
             '2009', '2010', '2011', '2012','2013', '2014', '2015', '2016', '2017', '2018']

states_simple = states_simple.select(bandList,intBandNames)


In [6]:
#Apply land cover change functions to images, first returns an image collection then converted to image
#where each band represents one year
lc_one_change_col = neighborhood_predictor_variables.get_year_stack_image_collection(
                                                                                states_simple,
                                                                                states_simple.bandNames().getInfo(), 
                                                                                band_indices=[0,1])
lc_one_change_col = lc_one_change_col.map(neighborhood_predictor_variables.lc_one_change)
lc_one_change_image = lc_one_change_col.toBands()
lc_one_change_image = lc_one_change_image.select(lc_one_change_image.bandNames(),
                                                 lc_one_change_col.aggregate_array('OriginalBand'))

lc_no_change_col = neighborhood_predictor_variables.get_year_stack_image_collection(
                                                                                states_simple,
                                                                                states_simple.bandNames().getInfo(), 
                                                                                band_indices=[0,1])
lc_no_change_col = lc_no_change_col.map(neighborhood_predictor_variables.lc_no_change)
lc_no_change_image = lc_no_change_col.toBands()
lc_no_change_image = lc_no_change_image.select(lc_no_change_image.bandNames(),
                                               lc_no_change_col.aggregate_array('OriginalBand'))

lc_reversed_col = neighborhood_predictor_variables.get_year_stack_image_collection(
                                                                            states_simple,
                                                                            states_simple.bandNames().getInfo(), 
                                                                            band_indices=[0,1,2])
lc_reversed_col = lc_reversed_col.map(neighborhood_predictor_variables.lc_reverse)
lc_reversed_image = lc_reversed_col.toBands()
lc_reversed_image = lc_reversed_image.select(lc_reversed_image.bandNames(), 
                                             lc_reversed_col.aggregate_array('OriginalBand'))

lc_changed_to_another_col = neighborhood_predictor_variables.get_year_stack_image_collection(
                                                                            states_simple,
                                                                            states_simple.bandNames().getInfo(), 
                                                                            band_indices=[0,1,2])
lc_changed_to_another_col = lc_changed_to_another_col.map(neighborhood_predictor_variables.lc_change_to_another)
lc_changed_to_another_image = lc_changed_to_another_col.toBands()
lc_changed_to_another_image = lc_changed_to_another_image.select(
                                                        lc_changed_to_another_image.bandNames(), 
                                                        lc_changed_to_another_col.aggregate_array('OriginalBand'))

lc_consistent_change_one_year_col = neighborhood_predictor_variables.get_year_stack_image_collection(
                                                                        states_simple,
                                                                        states_simple.bandNames().getInfo(), 
                                                                        band_indices=[0,1,2])
lc_consistent_change_one_year_col = lc_consistent_change_one_year_col.map(
                                        neighborhood_predictor_variables.lc_consistent_change_one_year)
lc_consistent_change_one_year_image = lc_consistent_change_one_year_col.toBands()
lc_consistent_change_one_year_image = lc_consistent_change_one_year_image.select(
                                        lc_consistent_change_one_year_image.bandNames(), 
                                        lc_consistent_change_one_year_col.aggregate_array('OriginalBand'))

lc_consistent_change_two_years_col = neighborhood_predictor_variables.get_year_stack_image_collection(
                                                                        states_simple,
                                                                        states_simple.bandNames().getInfo(), 
                                                                        band_indices=[0,1,2,3])
lc_consistent_change_two_years_col = lc_consistent_change_two_years_col.map(
                                        neighborhood_predictor_variables.lc_consistent_change_two_years)
lc_consistent_change_two_years_image = lc_consistent_change_two_years_col.toBands()
lc_consistent_change_two_years_image = lc_consistent_change_two_years_image.select(
                                        lc_consistent_change_two_years_image.bandNames(), 
                                        lc_consistent_change_two_years_col.aggregate_array('OriginalBand'))

lc_year_after_col = neighborhood_predictor_variables.get_year_stack_image_collection(
                                                        states_simple,
                                                        states_simple.bandNames().getInfo(), 
                                                        band_indices=[0,1])
lc_year_after_col = lc_year_after_col.map(neighborhood_predictor_variables.lc_year_after)
lc_year_after_image = lc_year_after_col.toBands()
lc_year_after_image = lc_year_after_image.select(lc_year_after_image.bandNames(), 
                                                 lc_year_after_col.aggregate_array('OriginalBand'))

year_image = ee.ImageCollection(lc_consistent_change_two_years_image.bandNames().map(
                                    lambda x: ee.Image.constant(ee.Number.parse(x)).updateMask(
                                    lc_one_change_image.select([x])))).toBands()
year_image = year_image.rename(lc_consistent_change_two_years_image.bandNames())

states_simple_masked = states_simple.select(lc_one_change_image.bandNames()).bandNames().map(
                                            lambda x: states_simple.select([x]).updateMask(
                                                lc_one_change_image.select([x])))
states_simple_masked = ee.ImageCollection(states_simple_masked).toBands()
states_simple_masked = states_simple_masked.rename(lc_one_change_image.bandNames().getInfo())



In [7]:
#print(year_image.bandNames().getInfo())

In [8]:
#Define kernel for neighborhood
kernel = ee.Kernel.fixed(3,3,
                         [[1,1,1],
                          [1,0,1],
                          [1,1,1]]
                          ,1,1)

#Other kernel options commented below:
kernel = ee.Kernel.gaussian(radius=1000, units='meters', sigma=1000)

kernel = ee.Kernel.circle(radius=17, units='pixels')
kernel = ee.Kernel.square(radius=1.5, units='pixels')

In [9]:
#Get neighborhood information on pixels
states_lc_no_change_neighbors = lc_no_change_image.convolve(kernel)
states_lc_one_change_neighbors = lc_one_change_image.convolve(kernel)
states_lc_reverse_neighbors = lc_reversed_image.convolve(kernel)
states_lc_change_to_another_neighbors = lc_changed_to_another_image.convolve(kernel)
states_lc_consistent_change_one_year_neighbors = lc_consistent_change_one_year_image.convolve(kernel)
states_lc_consistent_change_two_years_neighbors = lc_consistent_change_two_years_image.convolve(kernel)


In [10]:
#Define list of images and the corresponding names (which will be used as column names) to sample for covariates
image_list = [lc_consistent_change_two_years_image,
              lc_one_change_image,
              year_image,
              states_simple_masked, 
              lc_year_after_image,
              states_lc_no_change_neighbors,
              states_lc_one_change_neighbors,
              states_lc_reverse_neighbors,
              states_lc_change_to_another_neighbors,
              states_lc_consistent_change_one_year_neighbors]

image_name_list = ['consistent_change',
                   'change_occurred',
                   'year',
                   'current_state',
                   'year_after_state',
                   'no_lc_change_neighbors',
                   'one_lc_change_neighbors',
                   'lc_reverse_neighbors',
                   'lc_change_to_another_neighbors',
                   'lc_consistent_change_one_year_neighbors']

covariate_image_collection = ee.ImageCollection(image_list)
#Reproject images to original reprojection
#image_list = [x.reproject(crs=crs,crsTransform=crsTransform) for x in image_list]


In [11]:
training_points = ee.FeatureCollection('projects/wri-datalab/DynamicWorld_CD/Training_points_20200708')
validation_points = ee.FeatureCollection('projects/wri-datalab/DynamicWorld_CD/Validation_points_20200708')
predictors = ['current_state', 'lc_change_to_another_neighbors', 
              'lc_consistent_change_one_year_neighbors', 
              'lc_reverse_neighbors', 'no_lc_change_neighbors', 'one_lc_change_neighbors', 
              'year_after_state']
one_hot_features = ['current_state','year_after_state']
y_column = 'consistent_change'
xy = ['longitude','latitude']


In [12]:
#ee.Classifier.smileRandomForest(numberOfTrees, variablesPerSplit, minLeafPopulation, bagFraction, maxNodes, seed)
rf_classifier = gee_classifiers.train_rf_classifier(training_points, y_column, predictors)
print('RF accuracy: ', rf_classifier.confusionMatrix().accuracy().getInfo())
print(gee_classifiers.pretty_print_confusion_matrix(rf_classifier))




RF accuracy:  0.9330085261875761
                Predicted_False  Predicted_True
_                                              
Observed_False              474             350
Observed_True                35            4888


In [13]:
image_stack_2015 = covariate_image_collection.map(lambda x: x.select('2015')).toBands()
image_stack_2015 = image_stack_2015.rename(image_name_list)
image_stack_2015 = image_stack_2015.reproject(crs=crs,crsTransform=crsTransform)

covar_image_stack_2015 = image_stack_2015.select(predictors)
classified_image_2015 = covar_image_stack_2015.classify(rf_classifier)


In [14]:
#Map consistent change layer for one year to test if it works
Map3 = geemap.Map(center=[-9,-51], zoom=10)
Map3.addLayer(lc_consistent_change_two_years_image.updateMask(
                image_stack_2015.select('change_occurred')).select(['2015']),
                changeDetectionViz,name='2015 Consistent Change')
Map3.addLayer(classified_image_2015,changeDetectionViz,name='2015 Predicted')
display(Map3)


Map(center=[-9, -51], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=F…